In [1]:
from bs4 import BeautifulSoup 
import requests
import pymongo
from splinter import Browser
from splinter.exceptions import ElementDoesNotExist
import tweepy
from config import (consumer_key,
                    consumer_secret,
                    access_token,
                    access_token_secret)
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, parser=tweepy.parsers.JSONParser())
import pandas as pd
import pymongo
from pprint import pprint

In [2]:
!which chromedriver
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=True)

/usr/local/bin/chromedriver


In [3]:
# URL of page to be scraped
url = 'https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest'
browser.visit(url)
html = browser.html
# Retrieve page with the requests module
#response = requests.get(url)
# Create BeautifulSoup object; parse with 'lxml'
soup = BeautifulSoup(html, 'html.parser')
news_p_list = []
news_title_list = []

In [4]:
results = soup.find_all('li', class_ ='slide')

for result in results:
    newstitle = result.find('div',class_='content_title')
    news_title= newstitle.a.text
    news_p = result.find('div', class_ = 'article_teaser_body').text
    news_p_list.append(news_p)
    news_title_list.append(news_title)

    #print(news_title)
    #print("______")
    #print(news_p)
    #print("______")
print(news_title_list[0])
print(news_p_list[0])

Curiosity on the Move Again
NASA's Mars Curiosity rover drove about 197 feet over the weekend to a site called Lake Orcadie, pushing its total odometry to over 12 miles.


In [5]:
url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(url)
html = browser.html

soup = BeautifulSoup(html, 'html.parser')


In [6]:
results = soup.find('div',class_='carousel_items').a
print(soup.find('div',class_='carousel_items').article['style'].split("/",1)[1].rstrip("');"))
featured_image_url = "https://www.jpl.nasa.gov/" +  soup.find('div',class_='carousel_items').article['style'].split("/",1)[1].rstrip("');")
#featured_image_url = "https://www.jpl.nasa.gov" + results['data-fancybox-href']
print(featured_image_url)
#for result in results:
    #featured_image_url = result

    #print(featured_image_url)
#featured_image_url = https://www.jpl.nasa.gov/spaceimages/images/mediumsize/PIA18452_ip.jpg    

spaceimages/images/wallpaper/PIA18614-1920x1200.jpg
https://www.jpl.nasa.gov/spaceimages/images/wallpaper/PIA18614-1920x1200.jpg


In [10]:
tweets = api.user_timeline('MarsWxReport',count=1)
mars_weather = []
for tweet in tweets:
    pprint(tweet['text'])
    mars_weather.append(tweet['text'])
print(mars_weather)

('Sol 2224 (2018-11-08), high 0C/32F, low -72C/-97F, pressure at 8.65 hPa, '
 'daylight 06:19-18:36')
['Sol 2224 (2018-11-08), high 0C/32F, low -72C/-97F, pressure at 8.65 hPa, daylight 06:19-18:36']


In [58]:
target_url = 'https://space-facts.com/mars/'
tables = pd.read_html(target_url)
tables =tables[0]
tables = tables.rename(columns={0: "Info", 1:"Value"})
tables.set_index('Info', inplace=True)
weather_stuff = tables.to_dict()
weather_stuff = weather_stuff['Value']
weather_stuff

{'Equatorial Diameter:': '6,792 km',
 'Polar Diameter:': '6,752 km',
 'Mass:': '6.42 x 10^23 kg (10.7% Earth)',
 'Moons:': '2 (Phobos & Deimos)',
 'Orbit Distance:': '227,943,824 km (1.52 AU)',
 'Orbit Period:': '687 days (1.9 years)',
 'Surface Temperature:': '-153 to 20 °C',
 'First Record:': '2nd millennium BC',
 'Recorded By:': 'Egyptian astronomers'}

In [11]:
hemisphere_image_urls = [
    {"title": "Cerberus Hemisphere Enhanced", "img_url": "https://astrogeology.usgs.gov/cache/images/cfa62af2557222a02478f1fcd781d445_cerberus_enhanced.tif_full.jpg"},
    {"title": "Schiaparelli Hemisphere Enhanced", "img_url": "https://astrogeology.usgs.gov/cache/images/3cdd1cbf5e0813bba925c9030d13b62e_schiaparelli_enhanced.tif_full.jpg"},
    {"title": "Syrtis Major Hemisphere Enhanced", "img_url": "https://astrogeology.usgs.gov/cache/images/ae209b4e408bb6c3e67b6af38168cf28_syrtis_major_enhanced.tif_full.jpg"},
    {"title": "Valles Marineris Hemisphere Enhanced", "img_url": "https://astrogeology.usgs.gov/cache/images/7cf2da4bf549ed01c17f206327be4db7_valles_marineris_enhanced.tif_full.jpg"},
]


[{'title': 'Cerberus Hemisphere Enhanced', 'img_url': 'https://astrogeology.usgs.gov/cache/images/cfa62af2557222a02478f1fcd781d445_cerberus_enhanced.tif_full.jpg'}, {'title': 'Schiaparelli Hemisphere Enhanced', 'img_url': 'https://astrogeology.usgs.gov/cache/images/3cdd1cbf5e0813bba925c9030d13b62e_schiaparelli_enhanced.tif_full.jpg'}, {'title': 'Syrtis Major Hemisphere Enhanced', 'img_url': 'https://astrogeology.usgs.gov/cache/images/ae209b4e408bb6c3e67b6af38168cf28_syrtis_major_enhanced.tif_full.jpg'}, {'title': 'Valles Marineris Hemisphere Enhanced', 'img_url': 'https://astrogeology.usgs.gov/cache/images/7cf2da4bf549ed01c17f206327be4db7_valles_marineris_enhanced.tif_full.jpg'}]


In [12]:
scraped_dict = {"Mars_news_title":news_title_list[0],"Mars_news_p":news_p_list[0],"Mars_images":featured_image_url,"Mars_tweets":mars_weather,"Mars_stats": tables,"Mars_hemi": hemisphere_image_urls}
print(scraped_dict)

{'Mars_news_title': 'Curiosity on the Move Again', 'Mars_news_p': "NASA's Mars Curiosity rover drove about 197 feet over the weekend to a site called Lake Orcadie, pushing its total odometry to over 12 miles.", 'Mars_images': 'https://www.jpl.nasa.gov/spaceimages/images/wallpaper/PIA18614-1920x1200.jpg', 'Mars_tweets': ['Sol 2224 (2018-11-08), high 0C/32F, low -72C/-97F, pressure at 8.65 hPa, daylight 06:19-18:36'], 'Mars_stats': [                      0                              1
0  Equatorial Diameter:                       6,792 km
1       Polar Diameter:                       6,752 km
2                 Mass:  6.42 x 10^23 kg (10.7% Earth)
3                Moons:            2 (Phobos & Deimos)
4       Orbit Distance:       227,943,824 km (1.52 AU)
5         Orbit Period:           687 days (1.9 years)
6  Surface Temperature:                  -153 to 20 °C
7         First Record:              2nd millennium BC
8          Recorded By:           Egyptian astronomers], 'Mars_hemi': 